In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import norm
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
def coinToss(p,samples):
    prob = np.random.binomial(1,p,samples)
    print(prob)
    probCalc(prob,samples)
    return prob
def probCalc(prob,samples):
  succ=0
  fail=0
  for i in prob:
    if i>0.5:
      succ+=1
    else:
      fail+=1
  print(f"success = {succ/samples},failure = {fail/samples}")
#probability = 0.2
#samplespace = ["Heads", "Tails"]

#for i in range(10):
#    result = coinToss(probability)
#    print(f"{result} : {samplespace[result]}")

In [ ]:
# Probability of success
p = 0.5

In [ ]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(n_inputs=1):
 model = Sequential()
 model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
 model.add(Dense(1, activation='sigmoid'))
 # compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=1):
 model = Sequential()
 model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
 model.add(Dense(n_outputs, activation='linear'))
 return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
 # make weights in the discriminator not trainable
 discriminator.trainable = False
 # connect them
 model = Sequential()
 # add generator
 model.add(generator)
 # add the discriminator
 model.add(discriminator)
 # compile model
 model.compile(loss='binary_crossentropy', optimizer='adam')
 return model

# generate n real samples with class labels
def generate_real_samples(n):
 # generate inputs in [-0.5, 0.5]
 X = coinToss(p,n).reshape(n, 1)
 # generate class labels
 y = ones((n, 1))
 return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
 # generate points in the latent space
 x_input = np.random.uniform(0,1)#randn(latent_dim * n)
 # reshape into a batch of inputs for the network
 x_input = x_input.reshape(n, latent_dim)
 return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
 # generate points in latent space
 x_input = generate_latent_points(latent_dim, n)
 # predict outputs
 X = generator.predict(x_input)
 # create class labels
 y = zeros((n, 1))
 return X, y

# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
 # prepare real samples
 x_real, y_real = generate_real_samples(n)
 # evaluate discriminator on real examples
 _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
 # prepare fake examples
 x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
 # evaluate discriminator on fake examples
 _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
 # summarize discriminator performance
 print(f"epoch : {epoch},acc_real : {acc_real},acc_fake : {acc_fake}")
 #print(f"Fake Data : {x_fake}")
 probCalc(y_fake,n)
 # scatter plot real and fake data points
 xs = np.linspace(0, 1, n)
 pyplot.scatter(xs,x_real[:, 0] ,color='red')
 pyplot.scatter(xs,x_fake[:, 0], color='blue')
 pyplot.show()

# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=1000, n_batch=128, n_eval=2):
 # determine half the size of one batch, for updating the discriminator
 half_batch = int(n_batch / 2)
 # manually enumerate epochs
 for i in range(n_epochs):
  # prepare real samples
  x_real, y_real = generate_real_samples(half_batch)
  # prepare fake examples
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
  # update discriminator
  d_model.train_on_batch(x_real, y_real)
  d_model.train_on_batch(x_fake, y_fake)
  # prepare points in latent space as input for the generator
  x_gan = generate_latent_points(latent_dim, n_batch)
  # create inverted labels for the fake samples
  y_gan = ones((n_batch, 1))
  # update the generator via the discriminator's error
  gan_model.train_on_batch(x_gan, y_gan)
  # evaluate the model every n_eval epochs
 #if i % n_eval == 0:
  summarize_performance(i, g_model, d_model, latent_dim)


In [ ]:
# size of the latent space
latent_dim = 5
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

In [ ]:

# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(n_inputs=2):
 model = Sequential()
 model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
 model.add(Dense(1, activation='sigmoid'))
 # compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=2):
 model = Sequential()
 model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
 model.add(Dense(n_outputs, activation='linear'))
 return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
 # make weights in the discriminator not trainable
 discriminator.trainable = False
 # connect them
 model = Sequential()
 # add generator
 model.add(generator)
 # add the discriminator
 model.add(discriminator)
 # compile model
 model.compile(loss='binary_crossentropy', optimizer='adam')
 return model

# generate n real samples with class labels
def generate_real_samples(n):
 # generate inputs in [-0.5, 0.5]
 X1 = rand(n) - 0.5
 # generate outputs X^2
 X2 = X1 * X1
 # stack arrays
 X1 = X1.reshape(n, 1)
 X2 = X2.reshape(n, 1)
 X = hstack((X1, X2))
 # generate class labels
 y = ones((n, 1))
 return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
 # generate points in the latent space
 x_input = randn(latent_dim * n)
 # reshape into a batch of inputs for the network
 x_input = x_input.reshape(n, latent_dim)
 return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
 # generate points in latent space
 x_input = generate_latent_points(latent_dim, n)
 # predict outputs
 X = generator.predict(x_input)
 # create class labels
 y = zeros((n, 1))
 return X, y

# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
 # prepare real samples
 x_real, y_real = generate_real_samples(n)
 # evaluate discriminator on real examples
 _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
 # prepare fake examples
 x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
 # evaluate discriminator on fake examples
 _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
 # summarize discriminator performance
 print(epoch, acc_real, acc_fake)
 # scatter plot real and fake data points
 pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
 pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
 pyplot.show()

# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):
 # determine half the size of one batch, for updating the discriminator
 half_batch = int(n_batch / 2)
 # manually enumerate epochs
 for i in range(n_epochs):
  # prepare real samples
  x_real, y_real = generate_real_samples(half_batch)
  # prepare fake examples
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
  # update discriminator
  d_model.train_on_batch(x_real, y_real)
  d_model.train_on_batch(x_fake, y_fake)
  # prepare points in latent space as input for the generator
  x_gan = generate_latent_points(latent_dim, n_batch)
  # create inverted labels for the fake samples
  y_gan = ones((n_batch, 1))
  # update the generator via the discriminator's error
  gan_model.train_on_batch(x_gan, y_gan)
  # evaluate the model every n_eval epochs
 if (i+1) % n_eval == 0:
  summarize_performance(i, g_model, d_model, latent_dim)


In [ ]:
fake_data_test = generate_fake_samples(g_model,latent_dim,10)
print(fake_data_test)

NameError: ignored

In [ ]:
x_input = generate_latent_points(latent_dim, 50)
X = generator.predict(x_input)
print(X)
probCalc(X,50)

2/2 [==============================] - 0s 4ms/step
[[0.38757747]
 [0.30687594]
 [0.5681021 ]
 [0.5738385 ]
 [0.4710586 ]
 [0.2529512 ]
 [0.36022335]
 [0.27475578]
 [0.20818926]
 [0.6189085 ]
 [0.4801982 ]
 [0.72403   ]
 [0.35956973]
 [0.47859323]
 [0.788127  ]
 [0.6407723 ]
 [0.31979036]
 [0.3981322 ]
 [0.91831535]
 [0.93583506]
 [0.5001415 ]
 [0.51854855]
 [0.39308   ]
 [0.46189588]
 [0.21856573]
 [1.0743695 ]
 [0.20153987]
 [0.492158  ]
 [0.14833981]
 [0.3896194 ]
 [0.3322845 ]
 [0.2773993 ]
 [0.3226045 ]
 [1.0112678 ]
 [0.28989047]
 [0.6189178 ]
 [0.40670413]
 [1.0622622 ]
 [0.48672807]
 [0.24452353]
 [0.6483404 ]
 [0.53378147]
 [0.46546847]
 [0.1797174 ]
 [0.22505094]
 [0.23347384]
 [0.3918414 ]
 [0.65141475]
 [0.41192406]
 [0.41072732]]
success = 0.34,failure = 0.66


In [ ]:
torch.save(generator.state_dict(), "")

NameError: ignored